input files

In [ ]:
!unzip -o '/kaggle/input/jigsaw-toxic-comment-classification-challenge/*.zip' -d /kaggle/working > /dev/null

importing libraries

In [ ]:
import numpy as np
import pandas as pd
import torch
from torch.utils.data import TensorDataset, RandomSampler, SequentialSampler, DataLoader
from torch.nn import BCEWithLogitsLoss, Sigmoid
from tqdm.notebook import tqdm, trange
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
#from bs4 import BeautifulSoup
import re

other installations

In [ ]:
!pip install BeautifulSoup4

In [ ]:
! pip install pytorch_pretrained_bert pytorch-nlp -q

In [ ]:
from bs4 import BeautifulSoup
df_train = pd.read_csv("train.csv")
df_train.head()

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

In [ ]:
df_test = pd.read_csv("test.csv")
df_test.head()

In [ ]:
sample_submission = pd.read_csv("sample_submission.csv")
sample_submission.head()

In [ ]:
df_train.shape,df_test.shape,sample_submission.shape

check null values

In [ ]:
df_train.isnull().sum()

In [ ]:
df_test.isnull().sum()

text preprocessing

In [ ]:
def textPre(text):
    soup = BeautifulSoup(text, 'html.parser')
    text = re.sub('\[[^]]*\]', '', soup.get_text())
    pattern=r"[^a-zA-z0-9\s,']"
    text=re.sub(pattern,'',text)
    return text

df_train["comment_text"] = df_train["comment_text"].apply(textPre)
df_test["comment_text"] = df_test["comment_text"].apply(textPre)

In [ ]:
print('Processed sentence :'  , df_train['comment_text'][3])

Change the sentences for input to BERT

In [ ]:
df_train_sentence = df_train["comment_text"]
df_test_sentence = df_test["comment_text"]
df_train_sentence = ["[SCL] "+ i + " [SEP]"for i in df_train_sentence]
df_test_sentence = ["[SCL] "+ i + " [SEP]"for i in df_test_sentence]
df_train_sentence[0], df_test_sentence[0]

Tokenizing and mapping to the index

In [ ]:
from pytorch_pretrained_bert import BertTokenizer, BertForSequenceClassification, BertAdam
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case = True)

train_tokenizer_texts = list(map(lambda t: tokenizer.tokenize(t)[:510], tqdm(df_train_sentence)))

test_tokenizer_texts = list(map(lambda t: tokenizer.tokenize(t)[:510], tqdm(df_test_sentence)))

np.array(train_tokenizer_texts[0]), np.array(test_tokenizer_texts[0])

In [ ]:
MAX_LEN = 128

input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tqdm(train_tokenizer_texts)]
input_ids = pad_sequences(sequences = input_ids, maxlen = MAX_LEN, dtype = 'long', padding='post', truncating='post')

test_input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tqdm(test_tokenizer_texts)]
test_input_ids = pad_sequences(sequences = test_input_ids, maxlen = MAX_LEN, dtype = 'long', padding='post', truncating='post')


input_ids[0], test_input_ids[0]

In [ ]:
#Creating an attention mask - For actual tokens its set to 1, for padding tokens its set to 0
def create_attention_masks(input_ids):
    attention_masks = []
    for seq in tqdm(input_ids):
        seq_mask = [float(i>0) for i in seq]
        attention_masks.append(seq_mask)
    return np.array(attention_masks)

attention_masks = create_attention_masks(input_ids)
test_attention_masks = create_attention_masks(test_input_ids)
attention_masks[0], test_attention_masks[0]

In [ ]:
labels = df_train[["toxic","severe_toxic","obscene","threat","insult","identity_hate"]].to_numpy()
X_train, X_val, y_train, y_val = train_test_split(input_ids, labels, random_state = 123, test_size = 0.20)
attention_masks_train, attention_masks_val = train_test_split(attention_masks, random_state = 123, test_size = 0.20)

In [ ]:
X_train = torch.tensor(X_train)
X_val = torch.tensor(X_val)
y_train = torch.tensor(y_train) 
y_val = torch.tensor(y_val)
attention_masks_train = torch.tensor(attention_masks_train)
attention_masks_val = torch.tensor(attention_masks_val)

test_input_ids = torch.tensor(test_input_ids)
test_attention_masks = torch.tensor(test_attention_masks)

In [ ]:
BATCH_SIZE = 32
#Dataset wrapping tensors.
train_data = TensorDataset(X_train, attention_masks_train, y_train)
val_data = TensorDataset(X_val, attention_masks_val, y_val)
test_data = TensorDataset(test_input_ids, test_attention_masks)
#Samples elements randomly. If without replacement(default), then sample from a shuffled dataset.
train_sampler = RandomSampler(train_data)
val_sampler = SequentialSampler(val_data)
test_sampler = SequentialSampler(test_data)
#represents a Python iterable over a dataset
train_dataloader = DataLoader(train_data, sampler = train_sampler, batch_size = BATCH_SIZE)
val_dataloader = DataLoader(val_data, sampler = val_sampler, batch_size = BATCH_SIZE)
test_dataloader = DataLoader(test_data, sampler = test_sampler, batch_size = BATCH_SIZE)

In [ ]:
#Inititaing a BERT model
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels = 6)
model.cuda()

In [ ]:
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'gamma', 'beta']
optimizer_grouped_parameters = [
    {
        'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
        'weight_decay_rate': 0.01
    },
    {
        'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
        'weight_decay_rate': 0.0
    }
]

optimizer = BertAdam(optimizer_grouped_parameters, lr = 2e-5, warmup = .1)

In [ ]:
#freeing up memory
torch.cuda.empty_cache()
import gc
gc.collect()

In [ ]:
#Empty the GPU memory as it might be memory and CPU intensive while training
torch.cuda.empty_cache()
#Number of times the whole dataset will run through the network and model is fine-tuned
epochs = 2
#Iterate over number of epochs
for _ in trange(epochs, desc = "Epoch"):
    #Switch model to train phase where it will update gradients
    model.train()
    #Initaite train and validation loss, number of rows passed and number of batches passed
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0
    val_loss = 0
    nb_val_examples, nb_val_steps = 0, 0
    #Iterate over batches within the same epoch
    for batch in tqdm(train_dataloader):
        #Shift the batch to GPU for computation
        batch = tuple(t.to(device) for t in batch)
        #Load the input ids and masks from the batch
        b_input_ids, b_input_mask, b_labels = batch
        #Initiate gradients to 0 as they tend to add up
        optimizer.zero_grad()
        #Forward pass the input data
        logits = model(b_input_ids, token_type_ids = None, attention_mask = b_input_mask)
        #We will be using the Binary Cross entropy loss with added sigmoid function after that in BCEWithLogitsLoss
        loss_func = BCEWithLogitsLoss()
        #Calculate the loss between multilabel predicted outputs and actuals
        loss = loss_func(logits, b_labels.type_as(logits))
        #Backpropogate the loss and calculate the gradients
        loss.backward()
        #Update the weights with the calculated gradients
        optimizer.step()
        #Add the loss of the batch to the final loss, number of rows and batches
        tr_loss += loss.item()
        nb_tr_examples += b_input_ids.size(0)
        nb_tr_steps += 1
    #Print the current training loss 
    print("Train Loss: {}".format(tr_loss/nb_tr_examples))
    #Switch the model to evaluate stage at which the gradients wont be updated
    model.eval()
    #Iterate over the validation data
    for step, batch in enumerate(val_dataloader):
        #Shift the validation data to GPUs for computation
        batch = tuple(t.to(device) for t in batch)
        #We dont want to update the gradients
        with torch.no_grad():
            #Load the input ids and masks from the batch
            b_input_ids, b_input_mask, b_labels = batch
            #Forward pass the input data
            logits = model(b_input_ids, token_type_ids = None, attention_mask = b_input_mask)
            #We will be using the Binary Cross entropy loss with added sigmoid function after that in BCEWithLogitsLoss
            loss_func = BCEWithLogitsLoss()
            #Calculate the loss between multilabel predicted outputs and actuals
            loss = loss_func(logits, b_labels.type_as(logits))
            #Add the loss of the batch to the final loss, number of rows and batches
            val_loss += loss.item()
            nb_val_examples += b_input_ids.size(0)
            nb_val_steps += 1
    #Print the current validation loss     
    print("Valid Loss: {}".format(val_loss/nb_val_examples))

In [ ]:
outputs = []

#Iterate over the test_loader 
for step, batch in enumerate(test_dataloader):
        #Transfer batch to GPUs
        batch = tuple(t.to(device) for t in batch)
        #We dont need to update gradients as we are just predicting
        with torch.no_grad():
            #Bring up the next batch of input_texts and attention_masks 
            b_input_ids, b_input_mask = batch
            #Forward propogate the inputs and get output as logits
            logits = model(b_input_ids, token_type_ids = None, attention_mask = b_input_mask)
            #Pass the outputs through a sigmoid function to get the multi-label preditions
            s = Sigmoid()
            out = s(logits).to('cpu').numpy()    
            #Add the predictions for this batch to the final list
            outputs.extend(out)

df_test = pd.merge(df_test, sample_submission, on = "id")
#Assign the predictions to the toxic_output columns
df_test[["toxic","severe_toxic","obscene","threat","insult","identity_hate"]] = outputs
#Drop text data as it is not expected in the submission file
df_test.drop(["comment_text"], axis = 1, inplace = True)
#Saving the submission dataframe
df_test.to_csv("sample_submission.csv", index = False)

In [ ]:
sub = pd.read_csv('sample_submission.csv')
sub.head()